In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
i_X_p, i_X_0s = rig_dyn.forward_kinematics(floating_base = True)

In [3]:
H0 , R0, p0 = pluck.spatial_to_homogeneous(i_X_0s[0])
T0 = cs.vertcat(p0, quatT.rotation_matrix_to_quaternion(R0, order='wxyz'))

H1 , R1, p1 = pluck.spatial_to_homogeneous(i_X_0s[1])
T1 = cs.vertcat(p1, quatT.rotation_matrix_to_quaternion(R1, order='wxyz'))

H2 , R2, p2 = pluck.spatial_to_homogeneous(i_X_0s[2])
T2 = cs.vertcat(p2, quatT.rotation_matrix_to_quaternion(R2, order='wxyz'))

H3 , R3, p3 = pluck.spatial_to_homogeneous(i_X_0s[3])
T3 = cs.vertcat(p3, quatT.rotation_matrix_to_quaternion(R3, order='wxyz'))

H4 , R4, p4 = pluck.spatial_to_homogeneous(i_X_0s[4])
T4 = cs.vertcat(p4, quatT.rotation_matrix_to_quaternion(R4, order='wxyz'))

fk_eval = cs.Function("fkeval", [ss.q, ss.baseT_xyz, ss.baseT_rpy, ss.p_n], [T0, T1, T2, T3, T4])
fk_eval.save("fk_eval.casadi")

In [4]:
# forward kinematics wrt to NED origin
T_i = fk_eval([cs.pi, cs.pi, cs.pi, cs.pi],[0.140, 0.000, -0.120], [3.142, 0.000, 0.000],[0.2,1,0.5, 0,0,0])
T_i[4]

DM([0.34, 1.00013, 0.05495, -0.000176386, 0.866025, 0.500001, 0.000101837])

In [5]:
len(T_i)

5

In [6]:
# c , cpp or matlab code generation for forward dynamics
fk_eval.generate("fk_eval_.c")
os.system(f"gcc -fPIC -shared fk_eval_.c -o libFK.so")

0

In [5]:
# whole body jacobian
# J_H = cs.jacobian(p, cs.vertcat(ss.p_n, ss.q)) 